In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

In [2]:
data = pd.read_csv('data.csv')
data.head()

,Unnamed: 0,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Time_updated,Year,Month
0,1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,2012-09-06 17:00:00,2012,9
1,4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,2012-10-20 17:00:00,2012,10
2,5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,4,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...,2012-07-11 17:00:00,2012,7
3,6,7,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0,0,5,1340150400,Great! Just as good as the expensive brands!,This saltwater taffy had great flavors and was...,2012-06-19 17:00:00,2012,6
4,7,8,B006K2ZZ7K,A3JRGQVEQN31IQ,Pamela G. Williams,0,0,5,1336003200,"Wonderful, tasty taffy",This taffy is so good. It is very soft and ch...,2012-05-02 17:00:00,2012,5


In [3]:
data.dtypes

Unnamed: 0                 int64
Id                         int64
ProductId                 object
UserId                    object
ProfileName               object
HelpfulnessNumerator       int64
HelpfulnessDenominator     int64
Score                      int64
Time                       int64
Summary                   object
Text                      object
Time_updated              object
Year                       int64
Month                      int64
dtype: object

Define the helpfulness ratio and label

In [4]:
data['Helpfulness'] = data['HelpfulnessNumerator']/data['HelpfulnessDenominator']

when the helpfulness denominator is less than 10, we identify a review as helpful when it has greater than 2/3 helpfulness ratio, when the helpfulness denominator is greater than 10, we identify a review as helpful when the ratio is greater than 1/2

In [5]:
def target(data):
    if data['HelpfulnessDenominator'] >= 10 and data['Helpfulness'] > 0.5:
        return 1
    elif data['HelpfulnessDenominator'] < 10 and data['Helpfulness'] >= 0.66:
        return 1
    else:
        return 0
    
data['helpful'] = data.apply(target, axis = 1)
data['helpful'].value_counts()

0    150751
1     47313
Name: helpful, dtype: int64

lemmatization in order to make data more clean

In [6]:
import pandas as pd
import nltk
nltk.download('averaged_perceptron_tagger')
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer



corpus = list(data["Text"].values)



lemmatizer = WordNetLemmatizer()

def nltk2wn_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

def lemmatize_sentence(sentence):
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    wn_tagged = map(lambda x: (x[0], nltk2wn_tag(x[1])), nltk_tagged)
    res_words = []
    for word, tag in wn_tagged:
        if tag is None:            
            res_words.append(word)
        else:
            res_words.append(lemmatizer.lemmatize(word, tag))
    return " ".join(res_words)


corpus_lem=[]
for review in corpus:
    corpus_lem = corpus_lem + [lemmatize_sentence(review)]
    
data['text_lem'] = corpus_lem
data.head()


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,Unnamed: 0,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Time_updated,Year,Month,Helpfulness,helpful,text_lem
0,1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,2012-09-06 17:00:00,2012,9,NaN,0,Product arrive label as Jumbo Salted Peanuts ....
1,4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,2012-10-20 17:00:00,2012,10,NaN,0,Great taffy at a great price . There be a wide...
2,5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,4,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...,2012-07-11 17:00:00,2012,7,NaN,0,I get a wild hair for taffy and order this fiv...
3,6,7,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0,0,5,1340150400,Great! Just as good as the expensive brands!,This saltwater taffy had great flavors and was...,2012-06-19 17:00:00,2012,6,NaN,0,This saltwater taffy have great flavor and be ...
4,7,8,B006K2ZZ7K,A3JRGQVEQN31IQ,Pamela G. Williams,0,0,5,1336003200,"Wonderful, tasty taffy",This taffy is so good. It is very soft and ch...,2012-05-02 17:00:00,2012,5,NaN,0,This taffy be so good . It be very soft and ch...


remove stopwords and convert sentences into vectorizer

In [7]:
from nltk.corpus import stopwords, wordnet
stop=stopwords.words('english')
vectorizer = TfidfVectorizer(ngram_range=(1,2),
                             token_pattern=r'\b[a-zA-Z0-9]{2,}\b',
                             max_df=0.5,
                             min_df=1, stop_words=stop)



X = vectorizer.fit_transform(data["text_lem"])
y = data["helpful"].values

## try different models and compare accuracy

XGBoost

In [8]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split




x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = 0.2,random_state = 0)


weight = len(y_train[y_train==0]) / len(y_train[y_train==1])
weight 


classifier = XGBClassifier(scale_pos_weight=weight)
classifier.fit(x_train, y_train)

y_pred = classifier.predict(x_test)



from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[20906,  9383],
       [ 4447,  4877]], dtype=int64)

AUC for XGBoost

In [9]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,y_pred)

0.6066381718970117

Accuracy for XGBoost

In [10]:
(20906+4877)/(20906+4877+9383+4447)

0.6508721884229924

Precision for XGBoost

In [9]:
4877/(9383+4877)

0.3420056100981767

Logistic Regression

In [11]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(x_train, y_train)
y_pred = lr.predict(x_test)

confusion_matrix(y_test, y_pred)

F:\anaconda3\anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


array([[30010,   279],
       [ 7181,  2143]], dtype=int64)

AUC of Logistic Regression

In [12]:
roc_auc_score(y_test,y_pred)

0.6103128575106851

Accuracy of Logistic Regression

In [13]:
(30010+2143)/(30010+2143+7181+279)

0.8116779845000379

Precision for Logistic Regression

In [10]:
2143/(249+2143)

0.8959030100334449

Random Forest

In [14]:
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
rf=RandomForestClassifier()

#Train the model using the training sets y_pred=clf.predict(X_test)
rf.fit(x_train,y_train)

y_pred=rf.predict(x_test)


from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

F:\anaconda3\anaconda\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


array([[30201,    88],
       [ 6083,  3241]], dtype=int64)

AUC of Random Forest

In [15]:
roc_auc_score(y_test,y_pred)

0.6723461262113909

Accuracy of Random Forest

In [23]:
(30201+3241)/(30201+3241+88+6083)

0.8442178072854871

Precision of Random Forest

In [11]:
3241/(3241+88)

0.9735656353259237

Decision Tree

In [26]:
from sklearn.tree import DecisionTreeClassifier


dt=RandomForestClassifier()

#Train the model using the training sets y_pred=clf.predict(X_test)
dt.fit(x_train,y_train)

y_pred=dt.predict(x_test)


from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

F:\anaconda3\anaconda\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


array([[30200,    89],
       [ 6048,  3276]], dtype=int64)

AUC of Decision Tree

In [27]:
roc_auc_score(y_test,y_pred)

0.674206495445229

Accuracy of Decision Tree

In [24]:
(30206+3280)/(30206+3280+83+6044)

0.8453285537576049

Precision for Decision Tree

In [12]:
3276/(3276+89)

0.9735512630014859

KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier()

#Train the model using the training sets y_pred=clf.predict(X_test)
knn.fit(x_train,y_train)

y_pred=knn.predict(x_test)


from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

In [20]:
confusion_matrix(y_test, y_pred)

array([[27156,  3133],
       [ 5569,  3755]], dtype=int64)

AUC of KNN

In [25]:
roc_auc_score(y_test,y_pred)

0.6496436307217449

Accuracy of KNN

In [21]:
(27156+3755)/(27156+3755+3133+5569)

0.7803246409007144

Precision for KNN

In [13]:
3755/(3133+3755)

0.545150987224158